In [55]:
print('só um teste boy é só um teste  qsqqsq') 


só um teste boy é só um teste  qsqqsq


In [56]:
df1 = pd.read_csv('utilities_disconnection.csv',sep=';')
df2 = pd.read_csv('utilities_disconnection_planilha2.csv',sep=';')

In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618346 entries, 0 to 618345
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   client_id                618346 non-null  int64  
 1   regional                 618346 non-null  int64  
 2   site                     618346 non-null  int64  
 3   client_class             618346 non-null  int64  
 4   client_subclass          618346 non-null  int64  
 5   connection_type          618346 non-null  object 
 6   connection_status        618346 non-null  object 
 7   installment_value        618346 non-null  float64
 8   installment_status       247561 non-null  object 
 9   total_debt               618346 non-null  float64
 10  total_bills              618346 non-null  int64  
 11  disconnection_date       618346 non-null  object 
 12  disconnection_requested  618346 non-null  object 
 13  contractor_id            618346 non-null  int64  
 14  disc

In [66]:
df1.isna().sum()

client_id                  0
regional                   0
site                       0
client_class               0
client_subclass            0
connection_type            0
connection_status          0
installment_value          0
installment_status         0
total_debt                 0
total_bills                0
disconnection_date         0
disconnection_requested    0
contractor_id              0
disconnection_executed     0
collected                  0
biphasic connection        0
single-phase connection    0
three-phase connection     0
dtype: int64

In [67]:
# substituir valores nulos por -1

df1['installment_status'].fillna(-1, inplace=True)

In [60]:
pysqldf("""select connection_status, count(connection_status) from df1
where collected = 1
group by connection_status
""")

,connection_status,count(connection_status)
0,connected,264648
1,disconnected,3468


In [61]:
pysqldf("""select connection_status, count(connection_status) from df1
where collected = 0
group by connection_status
""")

,connection_status,count(connection_status)
0,connected,48373
1,disconnected,301857


In [62]:
pysqldf = lambda q: sqldf(q, globals())
pysqldf("""select * from df1
where disconnection_requested <> disconnection_executed
""")

,client_id,regional,site,client_class,client_subclass,connection_type,connection_status,installment_value,installment_status,total_debt,total_bills,disconnection_date,disconnection_requested,contractor_id,disconnection_executed,collected
0,4,2,22212,1,1,biphasic connection,disconnected,2172.29,complete,484.10,4,16/03/2017,pole,156,circuit_break,0
1,6,4,42491,1,1,three-phase connection,connected,1487.33,current,1816.14,2,15/03/2017,remote,272,circuit_break,1
2,20,3,32281,1,1,biphasic connection,connected,0.00,-1,736.01,4,20/03/2017,pole,189,circuit_break,1
3,29,1,12390,1,1,single-phase connection,disconnected,0.00,-1,750.18,2,28/03/2017,circuit_break,20,branch,0
4,31,2,22330,1,1,single-phase connection,disconnected,387.36,complete,567.35,7,17/03/2017,circuit_break,166,pole,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124864,157324,4,42401,1,1,three-phase connection,disconnected,0.00,-1,2257.08,10,28/02/2018,pole,272,circuit_break,0
124865,340348,4,42401,1,1,biphasic connection,disconnected,0.00,-1,357.57,1,28/02/2018,pole,272,circuit_break,0
124866,898374,4,42401,1,1,three-phase connection,disconnected,0.00,-1,752.32,7,28/02/2018,pole,272,circuit_break,0
124867,157337,4,42401,1,1,biphasic connection,disconnected,0.00,-1,312.56,3,28/02/2018,pole,272,circuit_break,0


In [63]:
pysqldf("""select regional, count(regional) from df1
where disconnection_requested <> disconnection_executed
group by regional
""")

,regional,count(regional)
0,1,51384
1,2,7319
2,3,15108
3,4,51058


In [64]:
pysqldf("""select regional, count(regional) from df1
where disconnection_requested = 'remote'
group by regional
""")

,regional,count(regional)
0,1,27177
1,2,228196
2,4,29


In [65]:
dummie_conn_DF = pd.get_dummies(df1['connection_type'])
df1 = pd.concat((df1,dummie_conn_DF), axis = 1)